# 1. 依赖

In [ ]:
# --- 导入依赖库 ---
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
import re
from urllib.parse import urljoin, urlparse
from datetime import datetime
from tqdm.notebook import tqdm
!pip install selenium webdriver-manager
!pip install selenium webdriver_manager pandas
# --- 🔧 用户配置区 (请在此处替换) ---
# 1. 目标大学名称 (用于文件名和日志)
UNIVERSITY_NAME = "HKU"  # 例如: "HKU"

# 2. 只有在此域名下的链接才会被进一步抓取 (防止爬虫跑偏到 Facebook/Youtube 等外链)
ALLOWED_DOMAIN = "hku.hk"    # 例如: "hku.hk"

# 3. 所有的 Master List 入口链接 (可以是多个)
PROGRAM_LIST_URLS = [
    "https://portal.hku.hk/tpg-admissions/programme-listing", 
    # "<<REPLACE_PROGRAM_LIST_URL_2>>",
]

# --- 全局爬虫配置 ---
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.9',
}
TIMEOUT = 15  # 请求超时秒数
MAX_RETRIES = 3 # 最大重试次数

print(f"✅ 配置已加载: 目标大学 THE UNIVERSITY OF HONGKONG] | 允许域名 [https://portal.hku.hk/tpg-admissions/programme-listing]")

# 2. Crawler

In [ ]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from urllib.parse import urljoin
from IPython.display import display, HTML
from concurrent.futures import ThreadPoolExecutor, as_completed

# ==============================================================================
# 🟢【用户配置区】在这里修改并发数量
# ==============================================================================
MAX_WORKERS = 8   # 建议设置 3-8。设置越大越快，但对电脑负载越高。默认 5。
# ==============================================================================

# 全局变量
base_url = "https://portal.hku.hk"
list_url = "https://portal.hku.hk/tpg-admissions/programme-listing"
total_start_time = time.time()

# --- 1. 浏览器驱动生成器 (默认开启无头模式) ---
def get_driver(headless=True):
    chrome_options = Options()
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    
    # ⬇️ 这一行控制是否后台运行。True=后台(看不见)，False=前台(看得见)
    if headless:
        chrome_options.add_argument("--headless") 
        
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
    # 优化：禁止加载图片，提升后台抓取速度
    prefs = {"profile.managed_default_content_settings.images": 2}
    chrome_options.add_experimental_option("prefs", prefs)
    
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    return driver

# --- 2. Phase 1: 快速获取列表 (单线程) ---
print(f"🚀 [Phase 1] 正在扫描任务列表...")
driver = get_driver(headless=True) # 列表抓取也静默运行
temp_links = []

try:
    driver.get(list_url)
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'a[href*="programme-details"]')))
    
    while True:
        elements = driver.find_elements(By.CSS_SELECTOR, 'a[href*="programme-details"]')
        for elem in elements:
            try:
                link = elem.get_attribute("href")
                raw_text = elem.text.strip()
                if link and raw_text:
                    full_link = urljoin(base_url, link)
                    lines = raw_text.split('\n')
                    prog_name = lines[1].strip() if len(lines) > 1 else lines[0].strip()
                    if not any(d['link'] == full_link for d in temp_links):
                        temp_links.append({"name": prog_name, "link": full_link})
            except: continue
        try:
            next_btns = driver.find_elements(By.XPATH, "//a[contains(text(), '»')]")
            if not next_btns: break
            btn = next_btns[0]
            if "disabled" in btn.find_element(By.XPATH, "./..").get_attribute("class"): break
            driver.execute_script("arguments[0].click();", btn)
            time.sleep(1)
        except: break
finally:
    driver.quit()

print(f"✅ [Phase 1] 完成！共锁定 {len(temp_links)} 个任务。")
print(f"🔥 [Phase 2] 启动 {MAX_WORKERS} 个并发窗口进行后台抓取...")
time.sleep(1)

# --- 3. Phase 2: 分布式抓取逻辑 (单个Worker) ---
def process_single_item(item):
    item_start = time.time() # 开始计时
    local_driver = get_driver(headless=True) # 强制后台运行
    
    result = {
        "学校代码": "HK001",
        "学校名称": "The University of Hong Kong",
        "项目名称": item['name'],
        "项目官网链接": item['link'],
        "项目opendate": "N/A",
        "项目deadline": "N/A",
        "项目申请链接": "N/A",
        "学生案例": "",
        "面试问题": ""
    }
    
    try:
        local_driver.get(item['link'])
        WebDriverWait(local_driver, 15).until(EC.presence_of_element_located((By.XPATH, "//*[contains(text(), 'Start Date')]")))
        
        # 抓取时间
        try:
            start_elem = local_driver.find_element(By.XPATH, "//*[contains(text(), 'Start Date')]/following-sibling::*")
            result["项目opendate"] = start_elem.text.strip()
        except: pass
        
        try:
            dead_elem = local_driver.find_element(By.XPATH, "//*[contains(text(), 'Deadline')]/following-sibling::*")
            result["项目deadline"] = dead_elem.text.strip().replace("\n", " | ")
        except: pass

        # 抓取申请链接 (精准ID定位法)
        try:
            apply_btn = local_driver.find_element(By.XPATH, "//a[contains(text(), 'Apply Now')]")
            main_window = local_driver.current_window_handle
            local_driver.execute_script("arguments[0].click();", apply_btn)
            time.sleep(2)
            
            windows_step1 = local_driver.window_handles
            if len(windows_step1) > 1:
                local_driver.switch_to.window(windows_step1[-1])
                try:
                    # 直接找 ID="a_application"
                    applying_btn = WebDriverWait(local_driver, 5).until(
                        EC.element_to_be_clickable((By.CSS_SELECTOR, "#a_application a"))
                    )
                    local_driver.execute_script("arguments[0].click();", applying_btn)
                    time.sleep(3)
                    
                    windows_step2 = local_driver.window_handles
                    if len(windows_step2) > len(windows_step1):
                        local_driver.switch_to.window(windows_step2[-1])
                        result["项目申请链接"] = local_driver.current_url
                        local_driver.close()
                        local_driver.switch_to.window(windows_step1[-1])
                    else:
                        result["项目申请链接"] = local_driver.current_url
                except:
                    # 兜底：如果没找到ID，就记录当前说明页链接
                    result["项目申请链接"] = local_driver.current_url
                
                local_driver.close()
                local_driver.switch_to.window(main_window)
            else:
                result["项目申请链接"] = apply_btn.get_attribute("href")
        except: pass
        
    except Exception as e:
        print(f"❌ 抓取失败: {item['name'][:10]} - {e}")
    finally:
        local_driver.quit() # 释放内存
        
    # 计算单条耗时
    duration = time.time() - item_start
    return result, duration

# --- 4. 线程池执行与监控 ---
full_details_data = []
completed_count = 0

with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    # 提交任务
    future_to_item = {executor.submit(process_single_item, item): item for item in temp_links}
    
    print(f"⏳ 任务队列已建立，正在全力运行中 (请留意下方进度)...")
    
    for future in as_completed(future_to_item):
        item = future_to_item[future]
        try:
            data, duration = future.result()
            full_details_data.append(data)
            completed_count += 1
            
            # 实时打印单条耗时
            print(f"[{completed_count}/{len(temp_links)}] ✅ 完成: {data['项目名称'][:20]}... | ⏱️ 耗时: {duration:.2f}s")
            
        except Exception as exc:
            print(f"❌ 任务异常: {item['name']} generated an exception: {exc}")

# --- 5. 结果可视化展示 ---
total_time = time.time() - total_start_time
print(f"\n" + "="*50)
print(f"🎉 全部结束！总耗时: {total_time/60:.2f} 分钟")
print(f"📊 成功获取: {len(full_details_data)} 条数据")
print("="*50)

if full_details_data:
    df_result = pd.DataFrame(full_details_data)
    
    cols = ["学校代码", "学校名称", "项目名称", "项目官网链接", "项目申请链接", "项目opendate", "项目deadline", "学生案例", "面试问题"]
    for c in cols:
        if c not in df_result.columns: df_result[c] = ""
    df_result = df_result[cols]
    
    pd.set_option('display.max_colwidth', None)
    
    def make_clickable(val):
        if val and str(val).startswith("http"):
            return f'<a target="_blank" href="{val}">点击跳转</a>'
        return val
        
    print("\n↓↓↓ 最终抓取结果预览 (点击链接验证) ↓↓↓")
    display(HTML(df_result.head(10).style.format({'项目官网链接': make_clickable, '项目申请链接': make_clickable}).to_html(escape=False)))

# 3.Storage

In [ ]:
import pandas as pd

# 1. 检查是否有数据
if 'full_details_data' in locals() and full_details_data:
    print(f"🔄 正在整理 {len(full_details_data)} 条数据...")
    
    # 2. 创建 DataFrame
    df = pd.DataFrame(full_details_data)
    
    # 3. 强制规定列顺序 (严格按照你的要求)
    target_columns = [
        "学校代码", 
        "学校名称", 
        "项目名称", 
        "项目官网链接", 
        "项目申请链接", 
        "项目opendate", 
        "项目deadline", 
        "学生案例", 
        "面试问题"
    ]
    
    # 4. 自动补全缺失的列 (比如'学生案例'这种空的)
    for col in target_columns:
        if col not in df.columns:
            df[col] = ""
            
    # 5. 重新排序和筛选
    df_final = df[target_columns]
    
    # 6. 定义文件名
    excel_filename = "HKU_Projects_Final.xlsx"
    
    # 7. 导出 Excel
    try:
        # index=False 表示不保存左边那一列 0,1,2,3 的索引号
        df_final.to_excel(excel_filename, index=False)
        print("=" * 50)
        print(f"✅ 成功导出 Excel 文件！")
        print(f"📂 文件名: {excel_filename}")
        print(f"📊 包含数据: {len(df_final)} 行")
        print("=" * 50)
        print("💡 提示：你可以在左侧文件列表中找到并下载该文件。")
        
    except ImportError:
        # 兜底方案：如果环境里没有安装 openpyxl 库，就存 CSV
        print("⚠️ 检测到环境缺少 Excel 支持库 (openpyxl)，自动切换为 CSV 格式...")
        csv_filename = "HKU_Projects_Final.csv"
        df_final.to_csv(csv_filename, index=False, encoding="utf-8-sig")
        print(f"✅ 已保存为 CSV 文件: {csv_filename} (可用 Excel 打开)")
    except Exception as e:
        print(f"❌ 导出失败: {e}")

    # 8. 最后确认一下数据样子
    print("\n↓↓↓ 最终文件内容预览 ↓↓↓")
    display(df_final.head())

else:
    print("❌ 错误：内存中找不到 'full_details_data'。请确认上面的爬虫代码已经运行完毕。")